In [1]:

!pip install beautifulsoup4
!pip install requests
!pip install fake_useragent
!pip install selenium
!pip install python-dotenv


  Using cached requests-2.28.2-py3-none-any.whl (62 kB)
  Using cached certifi-2022.12.7-py3-none-any.whl (155 kB)
  Using cached charset_normalizer-3.1.0-cp39-cp39-win_amd64.whl (97 kB)
     ---------------------------------------- 0.0/140.9 kB ? eta -:--:--
     -------------------------------------- 140.9/140.9 kB 8.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/50.5 kB ? eta -:--:--
     ---------------------------------------- 50.5/50.5 kB 1.3 MB/s eta 0:00:00
  Using cached importlib_resources-5.12.0-py3-none-any.whl (36 kB)
     ---------------------------------------- 0.0/6.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/6.5 MB ? eta -:--:--
     ---------------------------------------- 0.1/6.5 MB 1.1 MB/s eta 0:00:06
     -- ------------------------------------- 0.5/6.5 MB 4.0 MB/s eta 0:00:02
     ----- ---------------------------------- 1.0/6.5 MB 6.0 MB/s eta 0:00:01
     --------- ------------------------------ 1.5/6.5 MB 7.3 MB/s

In [2]:
import psutil
import shutil
from fake_useragent import UserAgent
from selenium import webdriver
import subprocess
import os


ua = UserAgent()

class Scrapper:

    driver = None
    chrome_data_folder = os.path.join(os.getcwd(), "ChromeData")
    debug_window_pid = None
    debug_window_port = 9222
    options = None

    def __init__(self):
        chrome_options = {
            '--disable-gpu': None,
            '--disable-dev-shm-usage': None,
            '--disable-setuid-sandbox': None,
            '--no-first-run': None,
            '--no-sandbox': None,
            '--no-zygote': None,
            '--disable-blink-features': 'AutomationControlled',
            'start-maximized': None,
            'disable-infobars': None,
            '--disable-extensions': None,
            '--remote-debugging-port': '9222',
            'window-size': '1920x1080',
            'lang': 'ko_KR',
            'user-agent': ua.random
        }
        self.options = webdriver.ChromeOptions()
        for option, value in chrome_options.items():
            if value is not None:
                self.options.add_argument(f"{option}={value}")
            else:
                self.options.add_argument(option)

    def start(self):
        if os.path.exists(self.chrome_data_folder):
            shutil.rmtree(self.chrome_data_folder)
            print('기존 캐쉬 삭제 완료')
        debug_window = subprocess.Popen([
            "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe",
            f"--remote-debugging-port={self.debug_window_port}",
            f"--user-data-dir={self.chrome_data_folder}"])
        self.debug_window_pid = debug_window.pid
        self.driver = webdriver.Chrome(options=self.options)
        self.driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {"source": """ Object.defineProperty(navigator, 'webdriver', { get: () => undefined }) """})
        return self

    def end(self):
        self.driver.quit()
        for proc in psutil.process_iter():
            if proc.pid == self.debug_window_pid and f"--remote-debugging-port={self.debug_window_port}" in proc.cmdline():
                proc.terminate()

In [ ]:
from collections import OrderedDict

notes = OrderedDict()

In [49]:
import time
from dotenv import load_dotenv
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import html2markdown


results = []
scrapper = Scrapper()
driver = scrapper.start().driver
load_dotenv(os.path.join(os.getcwd(), '.env'))
wait = WebDriverWait(driver, 10)


# LeetCode 로그인 페이지로 이동
driver.get('https://leetcode.com/accounts/github/login/?next=%2F')

while True:
    try:
        # 이메일과 비밀번호 입력
        email_field = wait.until(EC.visibility_of_element_located((By.ID,'login_field')))
        email_field.clear()

        email_field.send_keys(os.getenv('EMAIL'))

        password_field = wait.until(EC.visibility_of_element_located((By.ID,'password')))
        password_field.clear()

        password_field.send_keys(os.getenv('PASSWORD'))

        # 로그인 버튼 클릭
        login_button = wait.until(EC.element_to_be_clickable((By.NAME,"commit")))
        login_button.click()

        wait.until(lambda driver: "login" not in driver.current_url)
        break
    except Exception as e:
        print(e)
        continue

try:
    # 2초 대기
    time.sleep(2)
    got_one = False
    while not got_one:
        try:
            # 문제 페이지로 이동
            driver.get(f'https://leetcode.com/problemset/all/?page=1')
            time.sleep(10)
            # 문제 목록 가져오기
            random_link_ele = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR,'#__next > div > div.mx-auto.w-full.grow.p-4.md\:mt-0.md\:max-w-\[888px\].md\:p-6.lg\:max-w-screen-xl.mt-\[50px\] > div.grid.grid-cols-4.gap-4.md\:grid-cols-3.lg\:grid-cols-4.lg\:gap-6 > div.col-span-4.z-base.md\:col-span-2.lg\:col-span-3 > div:nth-child(7) > div.mb-3.flex.flex-col > div > div.ml-auto.inline-flex.shrink-0.items-center.pl-2\.5.cursor-pointer')))
            print("debug")
            driver.execute_script("arguments[0].click();", random_link_ele)

            link = driver.current_url

            title = driver.find_element(By.CSS_SELECTOR, "#qd-content > div.h-full.flex-col.ssg__qd-splitter-primary-w > div > div > div > div.flex.h-full.w-full.overflow-y-auto > div > div > div.w-full.px-5.pt-4 > div > div:nth-child(1) > div.flex-1 > div > div > span").text

            # 검색할 디렉토리 경로
            dir_path = "./problems/leetcode/"

            # 이미 폴더가 존재하는지 확인
            if os.path.exists(os.path.join(dir_path, title)):
                print(f"폴더 '{title}'이 이미 존재합니다.")
                continue
            else:
                # 폴더 생성
                got_one = True
                os.makedirs(os.path.join(dir_path, title))

            content = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "._1l1MA")))
            content = html2markdown.convert(content.get_attribute('innerHTML'))

            # README.md 파일 생성
            with open(os.path.join(dir_path, title, "README.md"), "w") as f:
                f.write(f"### {title}\n\n")
                f.write(f"[leetcode]({link})\n\n")
                f.write(f"{content}")

        except TimeoutError as e:
            print(e)
            continue

except Exception as e:
    print(e)
finally:
    scrapper.end()


기존 캐쉬 삭제 완료
Message: 

Message: 

Message: 

Message: invalid selector: An invalid or illegal selector was specified
  (Session info: chrome=111.0.5563.147)
Stacktrace:
Backtrace:
	(No symbol) [0x0056DCE3]
	(No symbol) [0x005039D1]
	(No symbol) [0x00414DA8]
	(No symbol) [0x004176FB]
	(No symbol) [0x004175D0]
	(No symbol) [0x00417850]
	(No symbol) [0x0043FDF8]
	(No symbol) [0x004403AB]
	(No symbol) [0x0046EE62]
	(No symbol) [0x0045AF14]
	(No symbol) [0x0046D57C]
	(No symbol) [0x0045ACC6]
	(No symbol) [0x00436F68]
	(No symbol) [0x004380CD]
	GetHandleVerifier [0x007E3832+2506274]
	GetHandleVerifier [0x00819794+2727300]
	GetHandleVerifier [0x0081E36C+2746716]
	GetHandleVerifier [0x00616690+617600]
	(No symbol) [0x0050C712]
	(No symbol) [0x00511FF8]
	(No symbol) [0x005120DB]
	(No symbol) [0x0051C63B]
	BaseThreadInitThunk [0x775500F9+25]
	RtlGetAppContainerNamedObjectPath [0x77707BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77707B8E+238]

